In [1]:
import csv
import json
import pymongo
import pandas as pd
from pandas import json_normalize
from py2neo import Node,Relationship,Graph,Path,Subgraph


In [2]:
client = pymongo.MongoClient(host='127.0.0.1', port=27017)
db = client['traffic']
traffic_li = db['traffic_road_status_li']
traffic_pset = db['traffic_status']
pois_set = db['pois']
neo4j_url = 'http://localhost:7474/'
user = 'traffic'
pwd = '123456'
graph = Graph(host='127.0.0.1', auth=('neo4j', '123456'))


In [3]:
status_li = ['未知','畅通','缓行','拥堵','严重拥堵']
# week = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
week = ['weekend','workday']


In [4]:
start_time_m = '2022-01-18 06:30:00'
end_time_m = '2022-01-18 10:30:00'
start_time_a = '2022-01-18 16:00:00'
end_time_a = '2022-01-18 20:30:00'

m_times = pd.date_range(start=pd.Timestamp(start_time_m).round('5T'), end=pd.Timestamp(end_time_m).round('5T'),
                        freq="5T")
a_times = pd.date_range(start=pd.Timestamp(start_time_a).round('5T'), end=pd.Timestamp(end_time_a).round('5T'),
                        freq="5T")

morning = m_times.strftime('%H:%M').to_list()
afternoon = a_times.strftime('%H:%M').to_list()
moment_times = morning + afternoon


In [5]:
days = pd.date_range(start=pd.Timestamp('2022-01-23'),end=pd.Timestamp('2022-01-24'))
days = days.strftime('%Y-%m-%d').to_list()
days

['2022-01-23', '2022-01-24']

In [6]:
moment_times


['06:30',
 '06:35',
 '06:40',
 '06:45',
 '06:50',
 '06:55',
 '07:00',
 '07:05',
 '07:10',
 '07:15',
 '07:20',
 '07:25',
 '07:30',
 '07:35',
 '07:40',
 '07:45',
 '07:50',
 '07:55',
 '08:00',
 '08:05',
 '08:10',
 '08:15',
 '08:20',
 '08:25',
 '08:30',
 '08:35',
 '08:40',
 '08:45',
 '08:50',
 '08:55',
 '09:00',
 '09:05',
 '09:10',
 '09:15',
 '09:20',
 '09:25',
 '09:30',
 '09:35',
 '09:40',
 '09:45',
 '09:50',
 '09:55',
 '10:00',
 '10:05',
 '10:10',
 '10:15',
 '10:20',
 '10:25',
 '10:30',
 '16:00',
 '16:05',
 '16:10',
 '16:15',
 '16:20',
 '16:25',
 '16:30',
 '16:35',
 '16:40',
 '16:45',
 '16:50',
 '16:55',
 '17:00',
 '17:05',
 '17:10',
 '17:15',
 '17:20',
 '17:25',
 '17:30',
 '17:35',
 '17:40',
 '17:45',
 '17:50',
 '17:55',
 '18:00',
 '18:05',
 '18:10',
 '18:15',
 '18:20',
 '18:25',
 '18:30',
 '18:35',
 '18:40',
 '18:45',
 '18:50',
 '18:55',
 '19:00',
 '19:05',
 '19:10',
 '19:15',
 '19:20',
 '19:25',
 '19:30',
 '19:35',
 '19:40',
 '19:45',
 '19:50',
 '19:55',
 '20:00',
 '20:05',
 '20:10',


In [7]:
times_temp = {'id':'','name':''}
times_nodes = []
k = 1
for i,t in enumerate(moment_times):
    for j, w in enumerate(week):
        tdict = times_temp.copy()
        tdict['id'] = k
        tdict['name'] = w + ' - ' + t
        times_nodes.append(tdict)
        k+=1
        
t_df = json_normalize(times_nodes)
t_df.to_csv('weektimes_nodes.csv', index=False)
t_df

,id,name
0,1,weekend - 06:30
1,2,workday - 06:30
2,3,weekend - 06:35
3,4,workday - 06:35
4,5,weekend - 06:40
...,...,...
203,204,workday - 20:20
204,205,weekend - 20:25
205,206,workday - 20:25
206,207,weekend - 20:30


In [8]:
times_nodes

[{'id': 1, 'name': 'weekend - 06:30'},
 {'id': 2, 'name': 'workday - 06:30'},
 {'id': 3, 'name': 'weekend - 06:35'},
 {'id': 4, 'name': 'workday - 06:35'},
 {'id': 5, 'name': 'weekend - 06:40'},
 {'id': 6, 'name': 'workday - 06:40'},
 {'id': 7, 'name': 'weekend - 06:45'},
 {'id': 8, 'name': 'workday - 06:45'},
 {'id': 9, 'name': 'weekend - 06:50'},
 {'id': 10, 'name': 'workday - 06:50'},
 {'id': 11, 'name': 'weekend - 06:55'},
 {'id': 12, 'name': 'workday - 06:55'},
 {'id': 13, 'name': 'weekend - 07:00'},
 {'id': 14, 'name': 'workday - 07:00'},
 {'id': 15, 'name': 'weekend - 07:05'},
 {'id': 16, 'name': 'workday - 07:05'},
 {'id': 17, 'name': 'weekend - 07:10'},
 {'id': 18, 'name': 'workday - 07:10'},
 {'id': 19, 'name': 'weekend - 07:15'},
 {'id': 20, 'name': 'workday - 07:15'},
 {'id': 21, 'name': 'weekend - 07:20'},
 {'id': 22, 'name': 'workday - 07:20'},
 {'id': 23, 'name': 'weekend - 07:25'},
 {'id': 24, 'name': 'workday - 07:25'},
 {'id': 25, 'name': 'weekend - 07:30'},
 {'id': 2

In [10]:
#创建时间节点
node_ls = []

for i in range(len(times_nodes)):
    k+=1
    node = Node('weektimes',**times_nodes[i])
    node_ls.append(node)
subgraph = Subgraph(node_ls, [])
tx = graph.begin() 
# graph.merge(subgraph)
graph.commit(tx)

In [ ]:
t_speeds = traffic_pset.distinct('speed')

In [ ]:
t_speeds =sorted(t_speeds,key=lambda x:int(x))
speed_dict = []
for i,speed in enumerate(t_speeds):
    sdict = {}
    sdict['id'] = i + 1
    sdict['speed'] = speed + 'km/h'
    speed_dict.append(sdict)
    
    
speed_df = json_normalize(speed_dict)
speed_df.to_csv('speeds.csv', index=False)

speed_df
    

In [15]:
road_idl = traffic_pset.distinct('road_id')

In [35]:
# roads = []
# already_rids =[] 
# for i,rid in enumerate(road_idl):
#     road = {}
#     for item in traffic_pset.find({'road_id':rid}):
#         item.pop('status')
#         item.pop('speed')
#         item.pop('time')
#         item.pop('weekday')
#         item.pop('date')
        
        
        
    len(road_idl)

14843

In [16]:
roads = []
already_rids = []
k = 1
for item in traffic_pset.find():
    if item['road_id'] not in already_rids:
        item.pop('status', 'nan')
        item.pop('time', 'nan')
        item.pop('weekday', 'nan')
        item.pop('date', 'nan')
        item.pop('speed', 'nan')

        roads.append(item)
        already_rids.append(item['road_id'])
        print(k)
        k += 1
    if k == 14844:
        break

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


2379
2380
2381
2382
2383
2384
2385
2386
2387
2388
2389
2390
2391
2392
2393
2394
2395
2396
2397
2398
2399
2400
2401
2402
2403
2404
2405
2406
2407
2408
2409
2410
2411
2412
2413
2414
2415
2416
2417
2418
2419
2420
2421
2422
2423
2424
2425
2426
2427
2428
2429
2430
2431
2432
2433
2434
2435
2436
2437
2438
2439
2440
2441
2442
2443
2444
2445
2446
2447
2448
2449
2450
2451
2452
2453
2454
2455
2456
2457
2458
2459
2460
2461
2462
2463
2464
2465
2466
2467
2468
2469
2470
2471
2472
2473
2474
2475
2476
2477
2478
2479
2480
2481
2482
2483
2484
2485
2486
2487
2488
2489
2490
2491
2492
2493
2494
2495
2496
2497
2498
2499
2500
2501
2502
2503
2504
2505
2506
2507
2508
2509
2510
2511
2512
2513
2514
2515
2516
2517
2518
2519
2520
2521
2522
2523
2524
2525
2526
2527
2528
2529
2530
2531
2532
2533
2534
2535
2536
2537
2538
2539
2540
2541
2542
2543
2544
2545
2546
2547
2548
2549
2550
2551
2552
2553
2554
2555
2556
2557
2558
2559
2560
2561
2562
2563
2564
2565
2566
2567
2568
2569
2570
2571
2572
2573
2574
2575
2576
2577
2578


4124
4125
4126
4127
4128
4129
4130
4131
4132
4133
4134
4135
4136
4137
4138
4139
4140
4141
4142
4143
4144
4145
4146
4147
4148
4149
4150
4151
4152
4153
4154
4155
4156
4157
4158
4159
4160
4161
4162
4163
4164
4165
4166
4167
4168
4169
4170
4171
4172
4173
4174
4175
4176
4177
4178
4179
4180
4181
4182
4183
4184
4185
4186
4187
4188
4189
4190
4191
4192
4193
4194
4195
4196
4197
4198
4199
4200
4201
4202
4203
4204
4205
4206
4207
4208
4209
4210
4211
4212
4213
4214
4215
4216
4217
4218
4219
4220
4221
4222
4223
4224
4225
4226
4227
4228
4229
4230
4231
4232
4233
4234
4235
4236
4237
4238
4239
4240
4241
4242
4243
4244
4245
4246
4247
4248
4249
4250
4251
4252
4253
4254
4255
4256
4257
4258
4259
4260
4261
4262
4263
4264
4265
4266
4267
4268
4269
4270
4271
4272
4273
4274
4275
4276
4277
4278
4279
4280
4281
4282
4283
4284
4285
4286
4287
4288
4289
4290
4291
4292
4293
4294
4295
4296
4297
4298
4299
4300
4301
4302
4303
4304
4305
4306
4307
4308
4309
4310
4311
4312
4313
4314
4315
4316
4317
4318
4319
4320
4321
4322
4323


5774
5775
5776
5777
5778
5779
5780
5781
5782
5783
5784
5785
5786
5787
5788
5789
5790
5791
5792
5793
5794
5795
5796
5797
5798
5799
5800
5801
5802
5803
5804
5805
5806
5807
5808
5809
5810
5811
5812
5813
5814
5815
5816
5817
5818
5819
5820
5821
5822
5823
5824
5825
5826
5827
5828
5829
5830
5831
5832
5833
5834
5835
5836
5837
5838
5839
5840
5841
5842
5843
5844
5845
5846
5847
5848
5849
5850
5851
5852
5853
5854
5855
5856
5857
5858
5859
5860
5861
5862
5863
5864
5865
5866
5867
5868
5869
5870
5871
5872
5873
5874
5875
5876
5877
5878
5879
5880
5881
5882
5883
5884
5885
5886
5887
5888
5889
5890
5891
5892
5893
5894
5895
5896
5897
5898
5899
5900
5901
5902
5903
5904
5905
5906
5907
5908
5909
5910
5911
5912
5913
5914
5915
5916
5917
5918
5919
5920
5921
5922
5923
5924
5925
5926
5927
5928
5929
5930
5931
5932
5933
5934
5935
5936
5937
5938
5939
5940
5941
5942
5943
5944
5945
5946
5947
5948
5949
5950
5951
5952
5953
5954
5955
5956
5957
5958
5959
5960
5961
5962
5963
5964
5965
5966
5967
5968
5969
5970
5971
5972
5973


7438
7439
7440
7441
7442
7443
7444
7445
7446
7447
7448
7449
7450
7451
7452
7453
7454
7455
7456
7457
7458
7459
7460
7461
7462
7463
7464
7465
7466
7467
7468
7469
7470
7471
7472
7473
7474
7475
7476
7477
7478
7479
7480
7481
7482
7483
7484
7485
7486
7487
7488
7489
7490
7491
7492
7493
7494
7495
7496
7497
7498
7499
7500
7501
7502
7503
7504
7505
7506
7507
7508
7509
7510
7511
7512
7513
7514
7515
7516
7517
7518
7519
7520
7521
7522
7523
7524
7525
7526
7527
7528
7529
7530
7531
7532
7533
7534
7535
7536
7537
7538
7539
7540
7541
7542
7543
7544
7545
7546
7547
7548
7549
7550
7551
7552
7553
7554
7555
7556
7557
7558
7559
7560
7561
7562
7563
7564
7565
7566
7567
7568
7569
7570
7571
7572
7573
7574
7575
7576
7577
7578
7579
7580
7581
7582
7583
7584
7585
7586
7587
7588
7589
7590
7591
7592
7593
7594
7595
7596
7597
7598
7599
7600
7601
7602
7603
7604
7605
7606
7607
7608
7609
7610
7611
7612
7613
7614
7615
7616
7617
7618
7619
7620
7621
7622
7623
7624
7625
7626
7627
7628
7629
7630
7631
7632
7633
7634
7635
7636
7637


9170
9171
9172
9173
9174
9175
9176
9177
9178
9179
9180
9181
9182
9183
9184
9185
9186
9187
9188
9189
9190
9191
9192
9193
9194
9195
9196
9197
9198
9199
9200
9201
9202
9203
9204
9205
9206
9207
9208
9209
9210
9211
9212
9213
9214
9215
9216
9217
9218
9219
9220
9221
9222
9223
9224
9225
9226
9227
9228
9229
9230
9231
9232
9233
9234
9235
9236
9237
9238
9239
9240
9241
9242
9243
9244
9245
9246
9247
9248
9249
9250
9251
9252
9253
9254
9255
9256
9257
9258
9259
9260
9261
9262
9263
9264
9265
9266
9267
9268
9269
9270
9271
9272
9273
9274
9275
9276
9277
9278
9279
9280
9281
9282
9283
9284
9285
9286
9287
9288
9289
9290
9291
9292
9293
9294
9295
9296
9297
9298
9299
9300
9301
9302
9303
9304
9305
9306
9307
9308
9309
9310
9311
9312
9313
9314
9315
9316
9317
9318
9319
9320
9321
9322
9323
9324
9325
9326
9327
9328
9329
9330
9331
9332
9333
9334
9335
9336
9337
9338
9339
9340
9341
9342
9343
9344
9345
9346
9347
9348
9349
9350
9351
9352
9353
9354
9355
9356
9357
9358
9359
9360
9361
9362
9363
9364
9365
9366
9367
9368
9369


10674
10675
10676
10677
10678
10679
10680
10681
10682
10683
10684
10685
10686
10687
10688
10689
10690
10691
10692
10693
10694
10695
10696
10697
10698
10699
10700
10701
10702
10703
10704
10705
10706
10707
10708
10709
10710
10711
10712
10713
10714
10715
10716
10717
10718
10719
10720
10721
10722
10723
10724
10725
10726
10727
10728
10729
10730
10731
10732
10733
10734
10735
10736
10737
10738
10739
10740
10741
10742
10743
10744
10745
10746
10747
10748
10749
10750
10751
10752
10753
10754
10755
10756
10757
10758
10759
10760
10761
10762
10763
10764
10765
10766
10767
10768
10769
10770
10771
10772
10773
10774
10775
10776
10777
10778
10779
10780
10781
10782
10783
10784
10785
10786
10787
10788
10789
10790
10791
10792
10793
10794
10795
10796
10797
10798
10799
10800
10801
10802
10803
10804
10805
10806
10807
10808
10809
10810
10811
10812
10813
10814
10815
10816
10817
10818
10819
10820
10821
10822
10823
10824
10825
10826
10827
10828
10829
10830
10831
10832
10833
10834
10835
10836
10837
10838
10839
1084

12047
12048
12049
12050
12051
12052
12053
12054
12055
12056
12057
12058
12059
12060
12061
12062
12063
12064
12065
12066
12067
12068
12069
12070
12071
12072
12073
12074
12075
12076
12077
12078
12079
12080
12081
12082
12083
12084
12085
12086
12087
12088
12089
12090
12091
12092
12093
12094
12095
12096
12097
12098
12099
12100
12101
12102
12103
12104
12105
12106
12107
12108
12109
12110
12111
12112
12113
12114
12115
12116
12117
12118
12119
12120
12121
12122
12123
12124
12125
12126
12127
12128
12129
12130
12131
12132
12133
12134
12135
12136
12137
12138
12139
12140
12141
12142
12143
12144
12145
12146
12147
12148
12149
12150
12151
12152
12153
12154
12155
12156
12157
12158
12159
12160
12161
12162
12163
12164
12165
12166
12167
12168
12169
12170
12171
12172
12173
12174
12175
12176
12177
12178
12179
12180
12181
12182
12183
12184
12185
12186
12187
12188
12189
12190
12191
12192
12193
12194
12195
12196
12197
12198
12199
12200
12201
12202
12203
12204
12205
12206
12207
12208
12209
12210
12211
12212
1221

13419
13420
13421
13422
13423
13424
13425
13426
13427
13428
13429
13430
13431
13432
13433
13434
13435
13436
13437
13438
13439
13440
13441
13442
13443
13444
13445
13446
13447
13448
13449
13450
13451
13452
13453
13454
13455
13456
13457
13458
13459
13460
13461
13462
13463
13464
13465
13466
13467
13468
13469
13470
13471
13472
13473
13474
13475
13476
13477
13478
13479
13480
13481
13482
13483
13484
13485
13486
13487
13488
13489
13490
13491
13492
13493
13494
13495
13496
13497
13498
13499
13500
13501
13502
13503
13504
13505
13506
13507
13508
13509
13510
13511
13512
13513
13514
13515
13516
13517
13518
13519
13520
13521
13522
13523
13524
13525
13526
13527
13528
13529
13530
13531
13532
13533
13534
13535
13536
13537
13538
13539
13540
13541
13542
13543
13544
13545
13546
13547
13548
13549
13550
13551
13552
13553
13554
13555
13556
13557
13558
13559
13560
13561
13562
13563
13564
13565
13566
13567
13568
13569
13570
13571
13572
13573
13574
13575
13576
13577
13578
13579
13580
13581
13582
13583
13584
1358

14786
14787
14788
14789
14790
14791
14792
14793
14794
14795
14796
14797
14798
14799
14800
14801
14802
14803
14804
14805
14806
14807
14808
14809
14810
14811
14812
14813
14814
14815
14816
14817
14818
14819
14820
14821
14822
14823
14824
14825
14826
14827
14828
14829
14830
14831
14832
14833
14834
14835
14836
14837
14838
14839
14840
14841
14842
14843


In [18]:
data_dict = roads
node_ls = []
k=0
for i in range(len(data_dict)):
    k+=1
    node = Node('road',**data_dict[i])
    node_ls.append(node)
#     if k%100==0:
#         subgraph = Subgraph(node_ls, [])
#         tx = graph.begin() 
#         tx.delete(subgraph)
#         graph.commit(tx)
#         node_ls = []
subgraph = Subgraph(node_ls, [])
tx = graph.begin() 
tx.create(subgraph)
graph.commit(tx)        


In [13]:
roads_df = json_normalize(roads)
roads_df.to_csv('roads_nodes.csv',index=False,encoding='utf_8_sig')
roads_df

,_id,name,direction,angle,lcodes,polyline,road_id,r_li_id
0,57440e1c797711ecb31cf8e4e3b06c4b,京港澳高速,六里桥附近,210,-5162,"116.310097,39.8845749;116.309349,39.884243",c5625ff15f974b2427632dc60463f53e,573c1ed8797711ec83e3f8e4e3b06c4b
1,57471b7a797711ecbf33f8e4e3b06c4b,３１９省道,从大苑村桥到政通路,314,-3549,"116.099701,39.7674408;116.100693,39.7671432;11...",9978c28d7b96fb3f780873cc7eb94721,573c1ed8797711ec83e3f8e4e3b06c4b
2,57474292797711ecb198f8e4e3b06c4b,西六环,从房山梨园桥到土城村东桥,83,"229,230,231,232,233,234,235,236,237,238,239,24...","116.108543,39.6993027;116.107185,39.6996574;11...",bc32fba0180e454c6c4b4b85ffe1483a,573c1ed8797711ec83e3f8e4e3b06c4b
3,57476980797711ec9612f8e4e3b06c4b,西六环,从土城村东桥到房山梨园桥,263,"-241,-240,-239,-238,-237,-236,-235,-234,-233,-...","116.181503,40.164566;116.181351,40.1645241;116...",249a547f725ec348c312ab5c86daeb80,573c1ed8797711ec83e3f8e4e3b06c4b
4,5747b7b0797711ecaa6cf8e4e3b06c4b,凯旋大街,从长虹西路到京周路,68,"450,451","116.141075,39.7233086;116.14119,39.7234802;116...",45bdebbdbde0c81196ff64941b9033b1,573c1ed8797711ec83e3f8e4e3b06c4b
...,...,...,...,...,...,...,...,...
14838,7e6bbefa835611ec870ef8e4e3b06c4b,德贤路辅路,从旧宫新桥到久敬庄路,96,"2260,2261,2262,2263,2264","116.430313,39.7806702;116.430367,39.7808495;11...",51bb4ddf5c374bb50c127ecf5c1fd038,7e63a86e835611eca6fef8e4e3b06c4b
14839,7fb53650835611ec9fdcf8e4e3b06c4b,德贤路辅路,从久敬庄路到庑殿路,112,2265,"116.424767,39.8205833;116.424446,39.8209457;11...",5721abdef679e18630869093b610db60,7fb312b8835611ecb6a6f8e4e3b06c4b
14840,7fcdc6ac835611ec99e9f8e4e3b06c4b,德贤路辅路,从庑殿路到榴乡桥,83,2266,"116.42205,39.8256531;116.422043,39.8260841;116...",bc7c76d40f95ca8ff79c4ee30e3d4aa0,7fb312b8835611ecb6a6f8e4e3b06c4b
14841,0e3f360183b611ecb571f8e4e3b06c4b,樊羊路,从宏盛路到丰台东路,90,"8990,8300,8301,8302","116.313934,39.794075;116.313927,39.798954;116....",5f81e8bb4184c2f80cec1e0a11589799,0e35731883b611ecacbdf8e4e3b06c4b


In [10]:
# 创建status节点
status_nodes = []
for i,date in enumerate(days): 
    for item in traffic_pset.find({'date':date}):
        item['weektime'] = week[i] +' - ' + item['time']
        status_nodes.append(item)
    print(i)

0
1


In [3]:
neo4j_url = 'http://localhost:7474/'
user = 'traffic'
pwd = '123456'
graph = Graph(host='127.0.0.1', auth=('neo4j', '123456'))


In [12]:
graph

Graph('bolt://127.0.0.1:7687')

In [13]:
node_ls = []
    
k = 0
for i in range(len(status_nodes)):
    k+=1
    node = Node('speed',**status_nodes[i])
    node_ls.append(node)
    if k%100000==0 or k==len(status_nodes):
        subgraph = Subgraph(node_ls, [])
        tx = graph.begin() 
        tx.create(subgraph)
        graph.commit(tx)
        node_ls = []
        print(k)

100000
200000
300000
400000
500000
600000
676304


In [14]:
status_nodes[0]

{'_id': 'd6ce447a7bd211ec9abdf8e4e3b06c4b',
 'name': '京港澳高速',
 'status': '2',
 'direction': '从六里桥长途汽车站到六里桥',
 'angle': '30',
 'speed': '51',
 'lcodes': '5162',
 'polyline': '116.306419,39.8827858;116.308784,39.8838501;116.310165,39.8844795',
 'time': '06:30',
 'weekday': 'Sun',
 'date': '2022-01-23',
 'road_id': 'b92ee71443c60f2536a04ba030be95b5',
 'r_li_id': 'd6ca4cbe7bd211eca432f8e4e3b06c4b',
 'weektime': 'weekend - 06:30'}

In [ ]:
# 创建week_status 关系：
k = 0
rels = []
for i, s in enumerate(status_nodes):
    k += 1
    r_node = graph.nodes.match('weektimes', name=s['weektime']).first()
    t_node = graph.nodes.match('speed', _id=s['_id']).first()

    rels.append(
        Relationship(
            r_node, "Average_Speed", t_node, **{
                'name': s['name'],
                'road_id': s['road_id'],
                'weektime': s['weektime'],
                'status': s['status'],
                'r_type': 'Average_Speed'
            }))
    if k%10000==0:
        A = Subgraph(relationships=rels)
        graph.create(A)
        rels = []
        print(k)
A = Subgraph(relationships=rels)
graph.create(A)

In [10]:
# 
df = pd.read_csv('./roads_nodes.csv')
js = df.to_json(orient='records')
roads = json.loads(js)
len(roads)

14843

In [19]:
node_ls = []
for i in range(len(roads)):
    k+=1
    node = Node('roads',**roads[i])
    node_ls.append(node)
subgraph = Subgraph(node_ls, [])
tx = graph.begin() 
tx.create(subgraph)
graph.commit(tx)


In [25]:
len(roads)

14843

In [31]:
times_nodes

[{'id': 1, 'name': 'Mon - 06:30'},
 {'id': 2, 'name': 'Tue - 06:30'},
 {'id': 3, 'name': 'Wed - 06:30'},
 {'id': 4, 'name': 'Thu - 06:30'},
 {'id': 5, 'name': 'Fri - 06:30'},
 {'id': 6, 'name': 'Sat - 06:30'},
 {'id': 7, 'name': 'Sun - 06:30'},
 {'id': 8, 'name': 'Mon - 06:35'},
 {'id': 9, 'name': 'Tue - 06:35'},
 {'id': 10, 'name': 'Wed - 06:35'},
 {'id': 11, 'name': 'Thu - 06:35'},
 {'id': 12, 'name': 'Fri - 06:35'},
 {'id': 13, 'name': 'Sat - 06:35'},
 {'id': 14, 'name': 'Sun - 06:35'},
 {'id': 15, 'name': 'Mon - 06:40'},
 {'id': 16, 'name': 'Tue - 06:40'},
 {'id': 17, 'name': 'Wed - 06:40'},
 {'id': 18, 'name': 'Thu - 06:40'},
 {'id': 19, 'name': 'Fri - 06:40'},
 {'id': 20, 'name': 'Sat - 06:40'},
 {'id': 21, 'name': 'Sun - 06:40'},
 {'id': 22, 'name': 'Mon - 06:45'},
 {'id': 23, 'name': 'Tue - 06:45'},
 {'id': 24, 'name': 'Wed - 06:45'},
 {'id': 25, 'name': 'Thu - 06:45'},
 {'id': 26, 'name': 'Fri - 06:45'},
 {'id': 27, 'name': 'Sat - 06:45'},
 {'id': 28, 'name': 'Sun - 06:45'},
 

In [11]:
#创建关系
rels = []
for i, r in enumerate(roads):
    for j, t in enumerate(times_nodes):
        c = {}
        c['road_id'] = r['road_id']
        c['name'] = t['name']
        rels.append(c)
        
    print(i)    


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069
2070
2071
2072
2073
2074
2075
2076
2077
2078
2079
2080
2081
2082
2083
2084
2085
2086
2087
2088
2089
2090
2091
2092
2093
2094
2095
2096
2097
2098
2099
2100
2101
2102
2103
2104
2105
2106
2107
2108
2109
2110
2111
2112
2113
2114
2115
2116
2117
2118
2119
2120
2121
2122
2123
2124
2125
2126
2127
2128
2129
2130
2131
2132
2133
2134
2135
2136
2137
2138
2139
2140
2141
2142
2143
2144
2145
2146
2147
2148
2149
2150
2151
2152
2153
2154
2155
2156
2157
2158
2159
2160
2161
2162
2163
2164
2165
2166
2167
2168
2169
2170
2171
2172
2173
2174
2175
2176
2177
2178
2179
2180
2181
2182
2183
2184
2185
2186
2187
2188
2189
2190
2191
2192
2193
2194
2195
2196
2197
2198
2199
2200
2201
2202
2203
2204
2205
2206
2207
2208
2209
2210
2211
2212
2213
2214
2215
2216
2217
2218
2219
2220
2221
2222
2223
2224
2225
2226
2227
2228
2229
2230
2231
2232
2233
2234
2235


3766
3767
3768
3769
3770
3771
3772
3773
3774
3775
3776
3777
3778
3779
3780
3781
3782
3783
3784
3785
3786
3787
3788
3789
3790
3791
3792
3793
3794
3795
3796
3797
3798
3799
3800
3801
3802
3803
3804
3805
3806
3807
3808
3809
3810
3811
3812
3813
3814
3815
3816
3817
3818
3819
3820
3821
3822
3823
3824
3825
3826
3827
3828
3829
3830
3831
3832
3833
3834
3835
3836
3837
3838
3839
3840
3841
3842
3843
3844
3845
3846
3847
3848
3849
3850
3851
3852
3853
3854
3855
3856
3857
3858
3859
3860
3861
3862
3863
3864
3865
3866
3867
3868
3869
3870
3871
3872
3873
3874
3875
3876
3877
3878
3879
3880
3881
3882
3883
3884
3885
3886
3887
3888
3889
3890
3891
3892
3893
3894
3895
3896
3897
3898
3899
3900
3901
3902
3903
3904
3905
3906
3907
3908
3909
3910
3911
3912
3913
3914
3915
3916
3917
3918
3919
3920
3921
3922
3923
3924
3925
3926
3927
3928
3929
3930
3931
3932
3933
3934
3935
3936
3937
3938
3939
3940
3941
3942
3943
3944
3945
3946
3947
3948
3949
3950
3951
3952
3953
3954
3955
3956
3957
3958
3959
3960
3961
3962
3963
3964
3965


5549
5550
5551
5552
5553
5554
5555
5556
5557
5558
5559
5560
5561
5562
5563
5564
5565
5566
5567
5568
5569
5570
5571
5572
5573
5574
5575
5576
5577
5578
5579
5580
5581
5582
5583
5584
5585
5586
5587
5588
5589
5590
5591
5592
5593
5594
5595
5596
5597
5598
5599
5600
5601
5602
5603
5604
5605
5606
5607
5608
5609
5610
5611
5612
5613
5614
5615
5616
5617
5618
5619
5620
5621
5622
5623
5624
5625
5626
5627
5628
5629
5630
5631
5632
5633
5634
5635
5636
5637
5638
5639
5640
5641
5642
5643
5644
5645
5646
5647
5648
5649
5650
5651
5652
5653
5654
5655
5656
5657
5658
5659
5660
5661
5662
5663
5664
5665
5666
5667
5668
5669
5670
5671
5672
5673
5674
5675
5676
5677
5678
5679
5680
5681
5682
5683
5684
5685
5686
5687
5688
5689
5690
5691
5692
5693
5694
5695
5696
5697
5698
5699
5700
5701
5702
5703
5704
5705
5706
5707
5708
5709
5710
5711
5712
5713
5714
5715
5716
5717
5718
5719
5720
5721
5722
5723
5724
5725
5726
5727
5728
5729
5730
5731
5732
5733
5734
5735
5736
5737
5738
5739
5740
5741
5742
5743
5744
5745
5746
5747
5748


7423
7424
7425
7426
7427
7428
7429
7430
7431
7432
7433
7434
7435
7436
7437
7438
7439
7440
7441
7442
7443
7444
7445
7446
7447
7448
7449
7450
7451
7452
7453
7454
7455
7456
7457
7458
7459
7460
7461
7462
7463
7464
7465
7466
7467
7468
7469
7470
7471
7472
7473
7474
7475
7476
7477
7478
7479
7480
7481
7482
7483
7484
7485
7486
7487
7488
7489
7490
7491
7492
7493
7494
7495
7496
7497
7498
7499
7500
7501
7502
7503
7504
7505
7506
7507
7508
7509
7510
7511
7512
7513
7514
7515
7516
7517
7518
7519
7520
7521
7522
7523
7524
7525
7526
7527
7528
7529
7530
7531
7532
7533
7534
7535
7536
7537
7538
7539
7540
7541
7542
7543
7544
7545
7546
7547
7548
7549
7550
7551
7552
7553
7554
7555
7556
7557
7558
7559
7560
7561
7562
7563
7564
7565
7566
7567
7568
7569
7570
7571
7572
7573
7574
7575
7576
7577
7578
7579
7580
7581
7582
7583
7584
7585
7586
7587
7588
7589
7590
7591
7592
7593
7594
7595
7596
7597
7598
7599
7600
7601
7602
7603
7604
7605
7606
7607
7608
7609
7610
7611
7612
7613
7614
7615
7616
7617
7618
7619
7620
7621
7622


9177
9178
9179
9180
9181
9182
9183
9184
9185
9186
9187
9188
9189
9190
9191
9192
9193
9194
9195
9196
9197
9198
9199
9200
9201
9202
9203
9204
9205
9206
9207
9208
9209
9210
9211
9212
9213
9214
9215
9216
9217
9218
9219
9220
9221
9222
9223
9224
9225
9226
9227
9228
9229
9230
9231
9232
9233
9234
9235
9236
9237
9238
9239
9240
9241
9242
9243
9244
9245
9246
9247
9248
9249
9250
9251
9252
9253
9254
9255
9256
9257
9258
9259
9260
9261
9262
9263
9264
9265
9266
9267
9268
9269
9270
9271
9272
9273
9274
9275
9276
9277
9278
9279
9280
9281
9282
9283
9284
9285
9286
9287
9288
9289
9290
9291
9292
9293
9294
9295
9296
9297
9298
9299
9300
9301
9302
9303
9304
9305
9306
9307
9308
9309
9310
9311
9312
9313
9314
9315
9316
9317
9318
9319
9320
9321
9322
9323
9324
9325
9326
9327
9328
9329
9330
9331
9332
9333
9334
9335
9336
9337
9338
9339
9340
9341
9342
9343
9344
9345
9346
9347
9348
9349
9350
9351
9352
9353
9354
9355
9356
9357
9358
9359
9360
9361
9362
9363
9364
9365
9366
9367
9368
9369
9370
9371
9372
9373
9374
9375
9376


10687
10688
10689
10690
10691
10692
10693
10694
10695
10696
10697
10698
10699
10700
10701
10702
10703
10704
10705
10706
10707
10708
10709
10710
10711
10712
10713
10714
10715
10716
10717
10718
10719
10720
10721
10722
10723
10724
10725
10726
10727
10728
10729
10730
10731
10732
10733
10734
10735
10736
10737
10738
10739
10740
10741
10742
10743
10744
10745
10746
10747
10748
10749
10750
10751
10752
10753
10754
10755
10756
10757
10758
10759
10760
10761
10762
10763
10764
10765
10766
10767
10768
10769
10770
10771
10772
10773
10774
10775
10776
10777
10778
10779
10780
10781
10782
10783
10784
10785
10786
10787
10788
10789
10790
10791
10792
10793
10794
10795
10796
10797
10798
10799
10800
10801
10802
10803
10804
10805
10806
10807
10808
10809
10810
10811
10812
10813
10814
10815
10816
10817
10818
10819
10820
10821
10822
10823
10824
10825
10826
10827
10828
10829
10830
10831
10832
10833
10834
10835
10836
10837
10838
10839
10840
10841
10842
10843
10844
10845
10846
10847
10848
10849
10850
10851
10852
1085

12262
12263
12264
12265
12266
12267
12268
12269
12270
12271
12272
12273
12274
12275
12276
12277
12278
12279
12280
12281
12282
12283
12284
12285
12286
12287
12288
12289
12290
12291
12292
12293
12294
12295
12296
12297
12298
12299
12300
12301
12302
12303
12304
12305
12306
12307
12308
12309
12310
12311
12312
12313
12314
12315
12316
12317
12318
12319
12320
12321
12322
12323
12324
12325
12326
12327
12328
12329
12330
12331
12332
12333
12334
12335
12336
12337
12338
12339
12340
12341
12342
12343
12344
12345
12346
12347
12348
12349
12350
12351
12352
12353
12354
12355
12356
12357
12358
12359
12360
12361
12362
12363
12364
12365
12366
12367
12368
12369
12370
12371
12372
12373
12374
12375
12376
12377
12378
12379
12380
12381
12382
12383
12384
12385
12386
12387
12388
12389
12390
12391
12392
12393
12394
12395
12396
12397
12398
12399
12400
12401
12402
12403
12404
12405
12406
12407
12408
12409
12410
12411
12412
12413
12414
12415
12416
12417
12418
12419
12420
12421
12422
12423
12424
12425
12426
12427
1242

13826
13827
13828
13829
13830
13831
13832
13833
13834
13835
13836
13837
13838
13839
13840
13841
13842
13843
13844
13845
13846
13847
13848
13849
13850
13851
13852
13853
13854
13855
13856
13857
13858
13859
13860
13861
13862
13863
13864
13865
13866
13867
13868
13869
13870
13871
13872
13873
13874
13875
13876
13877
13878
13879
13880
13881
13882
13883
13884
13885
13886
13887
13888
13889
13890
13891
13892
13893
13894
13895
13896
13897
13898
13899
13900
13901
13902
13903
13904
13905
13906
13907
13908
13909
13910
13911
13912
13913
13914
13915
13916
13917
13918
13919
13920
13921
13922
13923
13924
13925
13926
13927
13928
13929
13930
13931
13932
13933
13934
13935
13936
13937
13938
13939
13940
13941
13942
13943
13944
13945
13946
13947
13948
13949
13950
13951
13952
13953
13954
13955
13956
13957
13958
13959
13960
13961
13962
13963
13964
13965
13966
13967
13968
13969
13970
13971
13972
13973
13974
13975
13976
13977
13978
13979
13980
13981
13982
13983
13984
13985
13986
13987
13988
13989
13990
13991
1399

In [15]:
road2week_df = json_normalize(rels)


In [16]:
road2week_df.to_csv('road2week_df.csv', index=False)


In [ ]:
# 创建关系:
rels = []
for i, r in enumerate(roads):
    r_node = graph.nodes.match('roads', road_id=r['road_id']).first()
    for j, t in enumerate(times_nodes):
        t_node = graph.nodes.match('weektimes', name=t['name']).first()
        rels.append(Relationship(r_node, "week_time", t_node))


In [ ]:
len(rels)

In [8]:
A=Subgraph(relationships=rels)   
graph.create(A)

NameError: name 'rels' is not defined

# 重新创建节点

In [11]:
#station
df = pd.read_csv('station.csv')
data_dict = json.loads(df.to_json(orient='records'))
node_ls = []
for i in range(len(data_dict)):
    k+=1
    node = Node('station',**data_dict[i])
    node_ls.append(node)
subgraph = Subgraph(node_ls, [])
tx = graph.begin() 
tx.merge(subgraph,primary_label='station',primary_key='id')
graph.commit(tx)
data_dict


[{'id': 'BV10013454',
  'name': '苹果园',
  'lng': 116.178945,
  'lat': 39.925686,
  'cityName': '北京市',
  'lineNames': 'S1线|6号线',
  'lineID': '900000069871|110100023339'},
 {'id': 'BV11494889',
  'name': '金安桥',
  'lng': 116.162586,
  'lat': 39.923298,
  'cityName': '北京市',
  'lineNames': 'S1线|6号线|11号线',
  'lineID': '900000069871|110100023339|900000159716'},
 {'id': 'BV10813547',
  'name': '四道桥',
  'lng': 116.13401,
  'lat': 39.91603,
  'cityName': '北京市',
  'lineNames': 'S1线',
  'lineID': '900000069871'},
 {'id': 'BV10813541',
  'name': '桥户营',
  'lng': 116.125809,
  'lat': 39.912383,
  'cityName': '北京市',
  'lineNames': 'S1线',
  'lineID': '900000069871'},
 {'id': 'BV10813543',
  'name': '上岸',
  'lng': 116.122225,
  'lat': 39.905138,
  'cityName': '北京市',
  'lineNames': 'S1线',
  'lineID': '900000069871'},
 {'id': 'BV10813544',
  'name': '栗园庄',
  'lng': 116.123254,
  'lat': 39.89578,
  'cityName': '北京市',
  'lineNames': 'S1线',
  'lineID': '900000069871'},
 {'id': 'BV10813542',
  'name': '小园',
  

In [12]:

#station
df = pd.read_csv('metro_line.csv')
data_dict = json.loads(df.to_json(orient='records'))
node_ls = []
for i in range(len(data_dict)):
    k+=1
    node = Node('metro_line',**data_dict[i])
    node_ls.append(node)
subgraph = Subgraph(node_ls, [])
tx = graph.begin() 
tx.merge(subgraph,'metro_line','id')
graph.commit(tx)
data_dict

[{'id': 900000069871,
  'name': 'S1线',
  'cityName': '北京市',
  'st_station': '苹果园',
  'ed_station': '石厂'},
 {'id': 110100023110,
  'name': '1号线八通线',
  'cityName': '北京市',
  'st_station': '古城',
  'ed_station': '环球度假区'},
 {'id': 110100023098,
  'name': '2号线',
  'cityName': '北京市',
  'st_station': '积水潭',
  'ed_station': '西直门'},
 {'id': 110100023076,
  'name': '4号线大兴线',
  'cityName': '北京市',
  'st_station': '天宫院',
  'ed_station': '安河桥北'},
 {'id': 110100023100,
  'name': '5号线',
  'cityName': '北京市',
  'st_station': '宋家庄',
  'ed_station': '天通苑北'},
 {'id': 110100023339,
  'name': '6号线',
  'cityName': '北京市',
  'st_station': '潞城',
  'ed_station': '金安桥'},
 {'id': 110100023054,
  'name': '7号线',
  'cityName': '北京市',
  'st_station': '环球度假区',
  'ed_station': '北京西站'},
 {'id': 110100023114,
  'name': '8号线',
  'cityName': '北京市',
  'st_station': '瀛海',
  'ed_station': '朱辛庄'},
 {'id': 110100023116,
  'name': '9号线',
  'cityName': '北京市',
  'st_station': '郭公庄',
  'ed_station': '国家图书馆'},
 {'id': 110100023282,
  'n

In [13]:
#station with lines
df = pd.read_csv('adjoin_rela.csv')
data_dict = json.loads(df.to_json(orient='records'))
len(data_dict)

0

In [14]:
data_dict

[]

In [16]:
# 创建站与站之间的关系:
rels = []
for i, r in enumerate(data_dict):
    r_node = graph.nodes.match('station', id=r['pid1']).first()
    t_node = graph.nodes.match('station', id=r['pid2']).first()
    r['r_type'] = 'ADJOIN'
    rels.append(Relationship(r_node, "ADJOIN", t_node,**r))
A=Subgraph(relationships=rels)   
graph.merge(A)
    


In [17]:
# 创建站与线路之间的关系
#station
df = pd.read_csv('station.csv')
data_dict = json.loads(df.to_json(orient='records'))
rels = []
for i,s in enumerate(data_dict):
    r_node = graph.nodes.match('station', id=s['id'])
    print(len(r_node))
    r_node = r_node.first()
    lids = s['lineID'].split('|')
    for j,lid in enumerate(lids):
        t_node = graph.nodes.match('metro_line', id=int(lid)).first()
        print(r_node,t_node)
        rels.append(Relationship(r_node, "BELONG", t_node,**{'pid':s['id'],'lid':int(lid)}))
        
        
A=Subgraph(relationships=rels)   
graph.merge(A)
    
        
    

1
(_494:station {cityName: '\u5317\u4eac\u5e02', id: 'BV10013454', lat: 39.925686, lineID: '900000069871|110100023339', lineNames: 'S1\u7ebf|6\u53f7\u7ebf', lng: 116.178945, name: '\u82f9\u679c\u56ed'}) (_620:metro_line {cityName: '\u5317\u4eac\u5e02', ed_station: '\u77f3\u5382', id: 900000069871, name: 'S1\u7ebf', st_station: '\u82f9\u679c\u56ed'})
(_494:station {cityName: '\u5317\u4eac\u5e02', id: 'BV10013454', lat: 39.925686, lineID: '900000069871|110100023339', lineNames: 'S1\u7ebf|6\u53f7\u7ebf', lng: 116.178945, name: '\u82f9\u679c\u56ed'}) (_622:metro_line {cityName: '\u5317\u4eac\u5e02', ed_station: '\u91d1\u5b89\u6865', id: 110100023339, name: '6\u53f7\u7ebf', st_station: '\u6f5e\u57ce'})
1
(_582:station {cityName: '\u5317\u4eac\u5e02', id: 'BV11494889', lat: 39.923298, lineID: '900000069871|110100023339|900000159716', lineNames: 'S1\u7ebf|6\u53f7\u7ebf|11\u53f7\u7ebf', lng: 116.162586, name: '\u91d1\u5b89\u6865'}) (_620:metro_line {cityName: '\u5317\u4eac\u5e02', ed_station: 

1
(_572:station {cityName: '\u5317\u4eac\u5e02', id: 'BV10013519', lat: 39.910139, lineID: '110100033086', lineNames: '\u516b\u901a\u7ebf', lng: 116.576709, name: '\u53cc\u6865'}) (_626:metro_line {cityName: '\u5317\u4eac\u5e02', ed_station: '\u73af\u7403\u5ea6\u5047\u533a', id: 110100033086, name: '\u516b\u901a\u7ebf', st_station: '\u56db\u60e0'})
1
(_595:station {cityName: '\u5317\u4eac\u5e02', id: 'BV10004294', lat: 39.90909, lineID: '110100033086', lineNames: '\u516b\u901a\u7ebf', lng: 116.599002, name: '\u7ba1\u5e84'}) (_626:metro_line {cityName: '\u5317\u4eac\u5e02', ed_station: '\u73af\u7403\u5ea6\u5047\u533a', id: 110100033086, name: '\u516b\u901a\u7ebf', st_station: '\u56db\u60e0'})
1
(_67:station {cityName: '\u5317\u4eac\u5e02', id: 'BV10001118', lat: 39.906121, lineID: '110100033086', lineNames: '\u516b\u901a\u7ebf', lng: 116.618658, name: '\u516b\u91cc\u6865'}) (_626:metro_line {cityName: '\u5317\u4eac\u5e02', ed_station: '\u73af\u7403\u5ea6\u5047\u533a', id: 110100033086, 

1
(_265:station {cityName: '\u5317\u4eac\u5e02', id: 'BV10001047', lat: 39.933801, lineID: '110100023098', lineNames: '2\u53f7\u7ebf', lng: 116.434133, name: '\u4e1c\u56db\u5341\u6761'}) (_621:metro_line {cityName: '\u5317\u4eac\u5e02', ed_station: '\u897f\u76f4\u95e8', id: 110100023098, name: '2\u53f7\u7ebf', st_station: '\u79ef\u6c34\u6f6d'})
1
(_316:station {cityName: '\u5317\u4eac\u5e02', id: 'BV10004298', lat: 39.924499, lineID: '110100023098|110100023339', lineNames: '2\u53f7\u7ebf|6\u53f7\u7ebf', lng: 116.434584, name: '\u671d\u9633\u95e8'}) (_621:metro_line {cityName: '\u5317\u4eac\u5e02', ed_station: '\u897f\u76f4\u95e8', id: 110100023098, name: '2\u53f7\u7ebf', st_station: '\u79ef\u6c34\u6f6d'})
(_316:station {cityName: '\u5317\u4eac\u5e02', id: 'BV10004298', lat: 39.924499, lineID: '110100023098|110100023339', lineNames: '2\u53f7\u7ebf|6\u53f7\u7ebf', lng: 116.434584, name: '\u671d\u9633\u95e8'}) (_622:metro_line {cityName: '\u5317\u4eac\u5e02', ed_station: '\u91d1\u5b89\u68

(_332:station {cityName: '\u5317\u4eac\u5e02', id: 'BV10000252', lat: 39.943114, lineID: '110100023076|110100023116|900000062236', lineNames: '4\u53f7\u7ebf\u5927\u5174\u7ebf|9\u53f7\u7ebf|16\u53f7\u7ebf', lng: 116.32519, name: '\u56fd\u5bb6\u56fe\u4e66\u9986'}) (_630:metro_line {cityName: '\u5317\u4eac\u5e02', ed_station: '\u7389\u6e0a\u6f6d\u4e1c\u95e8', id: 900000062236, name: '16\u53f7\u7ebf', st_station: '\u5317\u5b89\u6cb3'})
1
(_415:station {cityName: '\u5317\u4eac\u5e02', id: 'BV10000255', lat: 39.957904, lineID: '110100023076', lineNames: '4\u53f7\u7ebf\u5927\u5174\u7ebf', lng: 116.32322, name: '\u9b4f\u516c\u6751'}) (_605:metro_line {cityName: '\u5317\u4eac\u5e02', ed_station: '\u5b89\u6cb3\u6865\u5317', id: 110100023076, name: '4\u53f7\u7ebf\u5927\u5174\u7ebf', st_station: '\u5929\u5bab\u9662'})
1
(_466:station {cityName: '\u5317\u4eac\u5e02', id: 'BV10000258', lat: 39.966956, lineID: '110100023076', lineNames: '4\u53f7\u7ebf\u5927\u5174\u7ebf', lng: 116.321367, name: '\u4eb

(_131:station {cityName: '\u5317\u4eac\u5e02', id: 'BV10001045', lat: 39.933592, lineID: '110100023100', lineNames: '5\u53f7\u7ebf', lng: 116.417156, name: '\u5f20\u81ea\u5fe0\u8def'}) (_610:metro_line {cityName: '\u5317\u4eac\u5e02', ed_station: '\u5929\u901a\u82d1\u5317', id: 110100023100, name: '5\u53f7\u7ebf', st_station: '\u5b8b\u5bb6\u5e84'})
1
(_159:station {cityName: '\u5317\u4eac\u5e02', id: 'BV10013443', lat: 39.940782, lineID: '110100023100|110100023096', lineNames: '5\u53f7\u7ebf|\u9996\u90fd\u673a\u573a\u7ebf', lng: 116.416884, name: '\u5317\u65b0\u6865'}) (_610:metro_line {cityName: '\u5317\u4eac\u5e02', ed_station: '\u5929\u901a\u82d1\u5317', id: 110100023100, name: '5\u53f7\u7ebf', st_station: '\u5b8b\u5bb6\u5e84'})
(_159:station {cityName: '\u5317\u4eac\u5e02', id: 'BV10013443', lat: 39.940782, lineID: '110100023100|110100023096', lineNames: '5\u53f7\u7ebf|\u9996\u90fd\u673a\u573a\u7ebf', lng: 116.416884, name: '\u5317\u65b0\u6865'}) (_615:metro_line {cityName: '\u5317

...

1
(_405:station {cityName: '\u6df1\u5733\u5e02', id: 'BV10249956', lat: 22.648442, lineID: '440300024060', lineNames: '3\u53f7\u7ebf/\u9f99\u5c97\u7ebf', lng: 114.208994, name: '\u6a2a\u5c97'}) (_635:metro_line {cityName: '\u6df1\u5733\u5e02', ed_station: '\u53cc\u9f99', id: 440300024060, name: '3\u53f7\u7ebf/\u9f99\u5c97\u7ebf', st_station: '\u798f\u4fdd'})
1
(_476:station {cityName: '\u6df1\u5733\u5e02', id: 'BV10249955', lat: 22.658995, lineID: '440300024060', lineNames: '3\u53f7\u7ebf/\u9f99\u5c97\u7ebf', lng: 114.218042, name: '\u6c38\u6e56'}) (_635:metro_line {cityName: '\u6df1\u5733\u5e02', ed_station: '\u53cc\u9f99', id: 440300024060, name: '3\u53f7\u7ebf/\u9f99\u5c97\u7ebf', st_station: '\u798f\u4fdd'})
1
(_155:station {cityName: '\u6df1\u5733\u5e02', id: 'BV10249954', lat: 22.673079, lineID: '440300024060', lineNames: '3\u53f7\u7ebf/\u9f99\u5c97\u7ebf', lng: 114.22431, name: '\u8377\u5773'}) (_635:metro_line {cityName: '\u6df1\u5733\u5e02', ed_station: '\u53cc\u9f99', id: 440

1
(_61:station {cityName: '\u6df1\u5733\u5e02', id: 'BV10877296', lat: 22.710764, lineID: '440300024074', lineNames: '4\u53f7\u7ebf/\u9f99\u534e\u7ebf', lng: 114.044796, name: '\u957f\u6e56'}) (_629:metro_line {cityName: '\u6df1\u5733\u5e02', ed_station: '\u725b\u6e56', id: 440300024074, name: '4\u53f7\u7ebf/\u9f99\u534e\u7ebf', st_station: '\u798f\u7530\u53e3\u5cb8'})
1
(_356:station {cityName: '\u6df1\u5733\u5e02', id: 'BV10877297', lat: 22.715172, lineID: '440300024074', lineNames: '4\u53f7\u7ebf/\u9f99\u534e\u7ebf', lng: 114.054771, name: '\u89c2\u6f9c'}) (_629:metro_line {cityName: '\u6df1\u5733\u5e02', ed_station: '\u725b\u6e56', id: 440300024074, name: '4\u53f7\u7ebf/\u9f99\u534e\u7ebf', st_station: '\u798f\u7530\u53e3\u5cb8'})
1
(_29:station {cityName: '\u6df1\u5733\u5e02', id: 'BV10877295', lat: 22.720557, lineID: '440300024074', lineNames: '4\u53f7\u7ebf/\u9f99\u534e\u7ebf', lng: 114.064878, name: '\u677e\u5143\u53a6'}) (_629:metro_line {cityName: '\u6df1\u5733\u5e02', ed_sta

(_548:station {cityName: '\u6df1\u5733\u5e02', id: 'BV10242465', lat: 22.772271, lineID: '440300024046|440300024056', lineNames: '6\u53f7\u7ebf|11\u53f7\u7ebf/\u673a\u573a\u7ebf', lng: 113.829631, name: '\u677e\u5c97'}) (_609:metro_line {cityName: '\u6df1\u5733\u5e02', ed_station: '\u798f\u7530', id: 440300024056, name: '11\u53f7\u7ebf/\u673a\u573a\u7ebf', st_station: '\u78a7\u5934'})
1
(_273:station {cityName: '\u6df1\u5733\u5e02', id: 'BV10527870', lat: 22.774648, lineID: '440300024046', lineNames: '6\u53f7\u7ebf', lng: 113.83765, name: '\u6eaa\u5934'}) (_618:metro_line {cityName: '\u6df1\u5733\u5e02', ed_station: '\u79d1\u5b66\u9986', id: 440300024046, name: '6\u53f7\u7ebf', st_station: '\u677e\u5c97'})
1
(_240:station {cityName: '\u6df1\u5733\u5e02', id: 'BV10249897', lat: 22.77792, lineID: '440300024046', lineNames: '6\u53f7\u7ebf', lng: 113.84831, name: '\u677e\u5c97\u516c\u56ed'}) (_618:metro_line {cityName: '\u6df1\u5733\u5e02', ed_station: '\u79d1\u5b66\u9986', id: 44030002404

(_489:station {cityName: '\u6df1\u5733\u5e02', id: 'BV10602483', lat: 22.523138, lineID: '440300024050', lineNames: '7\u53f7\u7ebf/\u897f\u4e3d\u7ebf', lng: 114.059823, name: '\u7687\u5c97\u6751'}) (_637:metro_line {cityName: '\u6df1\u5733\u5e02', ed_station: '\u592a\u5b89', id: 440300024050, name: '7\u53f7\u7ebf/\u897f\u4e3d\u7ebf', st_station: '\u897f\u4e3d\u6e56'})
1
(_409:station {cityName: '\u6df1\u5733\u5e02', id: 'BV10243619', lat: 22.521799, lineID: '440300024050', lineNames: '7\u53f7\u7ebf/\u897f\u4e3d\u7ebf', lng: 114.073972, name: '\u7687\u5c97\u53e3\u5cb8'}) (_637:metro_line {cityName: '\u6df1\u5733\u5e02', ed_station: '\u592a\u5b89', id: 440300024050, name: '7\u53f7\u7ebf/\u897f\u4e3d\u7ebf', st_station: '\u897f\u4e3d\u6e56'})
1
(_241:station {cityName: '\u6df1\u5733\u5e02', id: 'BV10249908', lat: 22.531374, lineID: '440300024050', lineNames: '7\u53f7\u7ebf/\u897f\u4e3d\u7ebf', lng: 114.083598, name: '\u8d64\u5c3e'}) (_637:metro_line {cityName: '\u6df1\u5733\u5e02', ed_sta

(_150:station {cityName: '\u6df1\u5733\u5e02', id: 'BV10672444', lat: 22.69367, lineID: '900000055265', lineNames: '10\u53f7\u7ebf', lng: 114.125805, name: '\u5e73\u6e56'}) (_638:metro_line {cityName: '\u6df1\u5733\u5e02', ed_station: '\u798f\u7530\u53e3\u5cb8', id: 900000055265, name: '10\u53f7\u7ebf', st_station: '\u53cc\u62e5\u8857'})
1
(_505:station {cityName: '\u6df1\u5733\u5e02', id: 'BV10672440', lat: 22.686028, lineID: '900000055265', lineNames: '10\u53f7\u7ebf', lng: 114.121064, name: '\u79be\u82b1'}) (_638:metro_line {cityName: '\u6df1\u5733\u5e02', ed_station: '\u798f\u7530\u53e3\u5cb8', id: 900000055265, name: '10\u53f7\u7ebf', st_station: '\u53cc\u62e5\u8857'})
1
(_245:station {cityName: '\u6df1\u5733\u5e02', id: 'BV10672443', lat: 22.675974, lineID: '900000055265', lineNames: '10\u53f7\u7ebf', lng: 114.12225, name: '\u534e\u5357\u57ce'}) (_638:metro_line {cityName: '\u6df1\u5733\u5e02', ed_station: '\u798f\u7530\u53e3\u5cb8', id: 900000055265, name: '10\u53f7\u7ebf', st_s

In [93]:
client = pymongo.MongoClient(host='localhost', port=27017)
db = client['traffic']
metro_station = db['metro_station_v2']
metro_line = db['metro_line_v2']





In [94]:
rels = []
for item in metro_line.find():
    lid = int(item['_id'])
    st = item['st']
    for s in st:
        r_node = graph.nodes.match('station', id=s['poiid']).first()
        t_node = graph.nodes.match('metro_line', id=lid).first()
        rels.append(Relationship(r_node, "BELONG", t_node,**{'pid':s['poiid'],'lid':int(lid)}))
        
A=Subgraph(relationships=rels)   
graph.merge(A)
    
        
        
        
                        
        
    

In [ ]:
# 

In [12]:
df = pd.read_csv('person.csv')
data_dict = json.loads(df.to_json(orient='records'))
data_dict

[{'id': 'AEAAAAAAG', 'count': 2},
 {'id': 'AEAAAAAFB', 'count': 2},
 {'id': 'AEAAAACFF', 'count': 2},
 {'id': 'AEAAAACHG', 'count': 2},
 {'id': 'AEAAAAEBF', 'count': 2},
 {'id': 'AEAAAAHFB', 'count': 2},
 {'id': 'AEAAAAHIC', 'count': 2},
 {'id': 'AEAAACAHB', 'count': 2},
 {'id': 'AEAAACBHI', 'count': 4},
 {'id': 'AEAAACCBI', 'count': 2},
 {'id': 'AEAAAFEIF', 'count': 2},
 {'id': 'AEAAAGHBG', 'count': 2},
 {'id': 'AEAAAJBGD', 'count': 2},
 {'id': 'AEAAAJECD', 'count': 4},
 {'id': 'AEAAAJEDA', 'count': 2},
 {'id': 'AEAABJHAC', 'count': 4},
 {'id': 'AEAAJAAAD', 'count': 2},
 {'id': 'AEAAJACDE', 'count': 2},
 {'id': 'AEAAJADGF', 'count': 4},
 {'id': 'AEAAJAFDB', 'count': 2},
 {'id': 'AEAAJAFFA', 'count': 2},
 {'id': 'AEAAJAHED', 'count': 2},
 {'id': 'AEAAJAJAB', 'count': 4},
 {'id': 'AEAAJBAFC', 'count': 2},
 {'id': 'AEAAJBAFI', 'count': 2},
 {'id': 'AEAAJBBEB', 'count': 2},
 {'id': 'AEAAJBCEH', 'count': 2},
 {'id': 'AEAAJBDBA', 'count': 4},
 {'id': 'AEAAJBDJA', 'count': 2},
 {'id': 'AEAAJ

In [13]:
node_ls = []
k=0
for i in range(len(data_dict)):
    k+=1
    node = Node('person',**data_dict[i])
    node_ls.append(node)
#     if k%100==0:
#         subgraph = Subgraph(node_ls, [])
#         tx = graph.begin() 
#         tx.delete(subgraph)
#         graph.commit(tx)
#         node_ls = []
subgraph = Subgraph(node_ls, [])
tx = graph.begin() 
tx.create(subgraph)
graph.commit(tx)        


In [14]:
df = pd.read_csv('route_record.csv')
data_dict = json.loads(df.to_json(orient='records'))
data_dict
node_ls = []
k=0
for i in range(len(data_dict)):
    k+=1
    node = Node('route_record',**data_dict[i])
    node_ls.append(node)
#     if k%100==0:
#         subgraph = Subgraph(node_ls, [])
#         tx = graph.begin() 
#         tx.delete(subgraph)
#         graph.commit(tx)
#         node_ls = []
subgraph = Subgraph(node_ls, [])
tx = graph.begin() 
tx.create(subgraph)
graph.commit(tx)        


In [10]:
## pois
k = 0
node_ls = []

for  poi in pois_set.find():
    k+=1
    poi['id'] = poi.pop('_id')
    node= Node('POI',**poi)
    node_ls.append(node)
    if k%10000==0 :
        subgraph = Subgraph(node_ls, [])
        tx = graph.begin() 
        tx.create(subgraph)
        graph.commit(tx)  
        node_ls = []
        print(k)

subgraph = Subgraph(node_ls, [])
tx = graph.begin() 
tx.create(subgraph)
graph.commit(tx)        


    
    

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000


In [4]:
sql = 'MATCH (s:station  )-[r:BELONG]->(l:metro_line) where s.name contains "五道口"  return s,r,l'
rdata = graph.run(sql).data()

In [18]:
rdata[0]['r'].labels()

frozenset({'metro_line', 'station'})

In [4]:
# sql = 'Match (s:station)-[r:BELONG]->(l:metro_line) where l.name contains "13号" and s.lineID Contains "|"  return s,r,l'
# rdata = graph.run(sql).data()
# [print(r) for r in rdata]

In [3]:
# sql ='Match (s:station)-[r:BELONG]->(l:metro_line) where l.name starts with  "4号" and s.lineID Contains "|" and l.cityName="北京市" match (s2:station)-[r2:BELONG]->(l2:metro_line) where s2.id=s.id  return s,r,l,s2,r2,l2'
# rdata = graph.run(sql).data()
# print(rdata)

In [5]:
# sql='Match (path:speed),(r:road),(w:weektimes) where path.name contains "成府路" and path.weektime="weekend - 08:00" and path.weektime=w.name and r.road_id=path.road_id return path,r,w'
# sql = 'Match (r:road)-[r1:Week_Time]->(w:weektimes)-[r2:Average_Speed]->(path:speed) where path.name contains "成府路" and path.weektime="weekend - 08:00" and path.weektime=w.name and r.road_id=path.road_id return r,r1,w,r2,path'
# rdata = graph.run(sql).data()
# [print(r) for r in rdata]

In [9]:
sql = 'MATCH (start:station{name:"五道口"}), (end:station{name:"大兴机场"}) CALL apoc.algo.dijkstra(start, end, "ADJOIN","distance") YIELD path, weight RETURN path, weight, [r in relationships(path)| r.ln] AS ln'
rdata = graph.run(sql).data()


In [8]:
# sql = 'MATCH (n:POI) where n.name contains "五道口" return n'
# rdata = graph.run(sql).data()
# rdata

In [6]:
rdata[0]

{'n': Node('POI', adcode='110108', address='展春园西路蓟鑫大厦2-3楼', adname='海淀区', business_area='五道口', citycode='010', cityname='北京市', id='B0FFI4LFG7', lat=39.988589, lng=116.340157, location='116.340157,39.988589', name='登巴客栈(北京五道口店)', pcode='110000', type='住宿服务;旅馆招待所;旅馆招待所', typecode='100200')}

In [10]:
value = '五道口'
sql = "MATCH (entity1) - [rel:ADJOIN|BELONG] -> (entity2)  WHERE entity1.name contains \"" + str(
            value) + "\" RETURN rel,entity2"
rdata = graph.run(sql).data()


[{'rel': ADJOIN(Node('station', cityName='北京市', id='BV10006886', lat=39.992894, lineID='110100033066', lineNames='13号线', lng=116.337742, name='五道口'), Node('station', cityName='北京市', id='BV10001218', lat=39.976476, lineID='110100023282', lineNames='10号线', lng=116.33996, name='知春路'), distance=1829, id=261, ln='13号线', n1='五道口', n2='知春路', no=3, pid1='BV10006886', pid2='BV10001218', r_type='ADJOIN'),
  'entity2': Node('station', cityName='北京市', id='BV10001218', lat=39.976476, lineID='110100023282', lineNames='10号线', lng=116.33996, name='知春路')},
 {'rel': ADJOIN(Node('station', cityName='北京市', id='BV10006886', lat=39.992894, lineID='110100033066', lineNames='13号线', lng=116.337742, name='五道口'), Node('station', cityName='北京市', id='BV10013516', lat=40.033007, lineID='110100033066', lineNames='13号线', lng=116.320193, name='上地'), distance=4966, id=262, ln='13号线', n1='上地', n2='五道口', no=4, pid1='BV10013516', pid2='BV10006886', r_type='ADJOIN'),
  'entity2': Node('station', cityName='北京市', id='BV10013

In [18]:
for i,item in enumerate(rdata):
    edata = dict(item['entity2'])
    label = list(item['entity2'].labels)[0]
    edata['type'] = label
    item['entity2'] = edata
    rdata[i] = item
rdata

[{'rel': ADJOIN(Node('station', cityName='北京市', id='BV10006886', lat=39.992894, lineID='110100033066', lineNames='13号线', lng=116.337742, name='五道口'), Node('station', cityName='北京市', id='BV10001218', lat=39.976476, lineID='110100023282', lineNames='10号线', lng=116.33996, name='知春路'), distance=1829, id=261, ln='13号线', n1='五道口', n2='知春路', no=3, pid1='BV10006886', pid2='BV10001218', r_type='ADJOIN'),
  'entity2': {'cityName': '北京市',
   'lng': 116.33996,
   'name': '知春路',
   'lineID': '110100023282',
   'id': 'BV10001218',
   'lat': 39.976476,
   'lineNames': '10号线',
   'type': 'station'}},
 {'rel': ADJOIN(Node('station', cityName='北京市', id='BV10006886', lat=39.992894, lineID='110100033066', lineNames='13号线', lng=116.337742, name='五道口'), Node('station', cityName='北京市', id='BV10013516', lat=40.033007, lineID='110100033066', lineNames='13号线', lng=116.320193, name='上地'), distance=4966, id=262, ln='13号线', n1='上地', n2='五道口', no=4, pid1='BV10013516', pid2='BV10006886', r_type='ADJOIN'),
  'entity2

In [17]:
list(rdata[0]['entity2'].labels)

['station']

In [131]:
# 湖北POI
db_hb = client['hubei']
hb_pois = db_hb['pois']

node_ls = []

k = 0
for item in hb_pois.find():
    k+=1
    item['id'] = item.pop('_id')
    node = Node('HB_POI',**item)
    node_ls.append(node)
    if k%100000==0:
        subgraph = Subgraph(node_ls, [])
        tx = graph.begin() 
        tx.create(subgraph)
        graph.commit(tx)
        node_ls = []
        print(k)
subgraph = Subgraph(node_ls, [])
tx = graph.begin() 
tx.create(subgraph)
graph.commit(tx)
node_ls = []
print(k)

100000
200000
300000
372607


In [ ]:
for tr in tri